<a href="https://colab.research.google.com/github/skssushil/Web-mining/blob/master/Term_Document_matrix_and_Cosine_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import numpy as np
import regex as re
import math
import nltk
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from google.colab import files
try:
    from googlesearch import search
except ImportError:
    print("No module named 'google' found")
# to search
query =["tutorial on datastructure", "data science", "web mining"," Artificial intelegence"]
url=[]
for i in query:
  q=[]
  for j in search(i, tld="com", num=5, stop=10, pause=2):
    q.append(j)
  url.append(q)
print(url)

[['https://www.tutorialspoint.com/data_structures_algorithms/index.htm', 'https://www.tutorialspoint.com/data_structures_algorithms/data_structures_basics.htm', 'https://www.tutorialspoint.com/data_structures_algorithms/algorithms_basics.htm', 'https://www.tutorialspoint.com/data_structures_algorithms/graph_data_structure.htm', 'https://www.tutorialspoint.com/data_structures_algorithms/data_structure_overview.htm', 'https://www.javatpoint.com/data-structure-tutorial', 'https://www.javatpoint.com/data-structure-interview-questions', 'https://www.javatpoint.com/data-structure-introduction', 'https://www.javatpoint.com/singly-linked-list', 'https://www.javatpoint.com/data-structure-array'], ['https://www.kainos.com/my-experience-as-a-data-scientist', 'https://en.wikipedia.org/wiki/Data_science', 'https://en.wikipedia.org/wiki/Master_in_Data_Science', 'https://en.wikipedia.org/wiki/Big_data', 'https://en.wikipedia.org/wiki/Data_analysis', 'https://en.wikipedia.org/wiki/Unstructured_data', 

In [3]:
f_text=" "

In [4]:
Length=len(url)

In [5]:
Length1=len(url[0])

In [6]:
dataset=[]
for i in range(Length):
  for j in range(Length1):
    req = urllib.request.Request(url[i][j], headers={'User-Agent': 'Mozilla/5.0'})
    data = urllib.request.urlopen(req).read()
    html= data.decode()
    soup = BeautifulSoup(html,features="lxml")
    for script in soup(["script", "style"]):
      script.extract()
    # get text
    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
  f_text=f_text+text
  dataset.append(f_text)

In [7]:
M=len(dataset)

Pre Processing

In [8]:
def convert_lower_case(data):
  return np.char.lower(data)

In [9]:
def remove_string_special_characters(data):
    # removes special characters with ' '
    stripped = re.sub('[^a-zA-z\s]', '', data)
    stripped = re.sub('_', '', stripped)

    # Change any white space to one space
    stripped = re.sub('\s+', ' ', stripped)

    # Remove start and end white spaces
    stripped = stripped.strip()
    return stripped.lower()


In [10]:
def stemming(data):
    stemmer= PorterStemmer()    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [11]:
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [12]:
# Stopword removal
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [13]:
def preprocess(data):
  Clean_data= remove_string_special_characters(data)  # removing special character
  Clean_data=stemming(Clean_data)
  Clean_data= convert_numbers(Clean_data)
  Clean_data=remove_stop_words(Clean_data)
  return Clean_data
# preprocessing of data completed

In [14]:
processed_text1 = []
for i in dataset[:M]:
    processed_text1.append(str(preprocess(i)))

In [15]:
print(len(processed_text1))

4


In [16]:
df = pd.DataFrame({'Doc1': [processed_text1[0]],'Doc2': [processed_text1[1]],'Doc3': [processed_text1[2]],'Doc4': [processed_text1[3]]})

In [17]:
print(df)

                                                Doc1  ...                                               Doc4
0   ds array javatpoint home data structur java s...  ...   ds array javatpoint home data structur java s...

[1 rows x 4 columns]


In [18]:
# Initialize
vectorizer = TfidfVectorizer()
doc_vec = vectorizer.fit_transform(df.iloc[0])

In [19]:
# Create dataFrame
df2 = pd.DataFrame(doc_vec.toarray().transpose(),
                   index=vectorizer.get_feature_names())

In [20]:
# Change column headers
df2.columns = df.columns

In [21]:
df3=pd.DataFrame(vectorizer.get_feature_names())

In [22]:
print(df2.head(50))

                   Doc1      Doc2      Doc3      Doc4
aapl           0.000000  0.000000  0.000000  0.005142
abandon        0.000000  0.000000  0.004487  0.004054
abil           0.000000  0.000000  0.008975  0.012162
abl            0.000000  0.000000  0.000000  0.005142
aboul          0.000000  0.000000  0.004487  0.004054
abov           0.010431  0.007742  0.011881  0.013417
absolut        0.000000  0.000000  0.008975  0.008108
abstract       0.000000  0.000000  0.004487  0.004054
abstractrec    0.000000  0.000000  0.004487  0.004054
academi        0.000000  0.018939  0.007266  0.009846
acceler        0.000000  0.009470  0.003633  0.003282
accept         0.010431  0.007742  0.005940  0.005367
access         0.062588  0.046452  0.035642  0.032201
accessshop     0.000000  0.000000  0.004487  0.004054
accident       0.000000  0.000000  0.004487  0.004054
accomplish     0.000000  0.000000  0.004487  0.004054
accord         0.000000  0.000000  0.008975  0.008108
account        0.000000  0.0

In [23]:
url_list=[*url[0],*url[1],*url[2],*url[3]]

In [24]:
doc=[]
for k in url_list:
  req = urllib.request.Request(k, headers={'User-Agent': 'Mozilla/5.0'})
  data = urllib.request.urlopen(req).read()
  html= data.decode()
  soup = BeautifulSoup(html,features="lxml")
  for script in soup(["script", "style"]):
    script.extract()
  # get text
  text = soup.get_text()
  # break into lines and remove leading and trailing space on each
  lines = (line.strip() for line in text.splitlines())
  # break multi-headlines into a line each
  chunks = (phrase.strip() for line in lines for phrase in line.split(" "))
  # drop blank lines
  text = '\n'.join(chunk for chunk in chunks if chunk)
  doc.append(text)

In [25]:
N=len(doc)

In [26]:
processed_text = []
for i in doc[:N]:
    processed_text.append(word_tokenize(str(preprocess(i))))

In [27]:
len(processed_text)

40

In [28]:
DF = {}
for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])

In [29]:
total_vocab_size = len(DF)
print(total_vocab_size)

9767


In [30]:
total_vocab = [x for x in DF]

In [31]:
print(total_vocab[:20])

['data', 'structur', 'algorithm', 'tutori', 'tutorialspoint', 'home', 'job', 'tool', 'code', 'ground', 'current', 'affair', 'upsc', 'note', 'onlin', 'tutor', 'whiteboard', 'net', 'meet', 'tutorix']


In [32]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [33]:
doc = 0
tf_idf = {}
for i in range(N):    
    tokens = processed_text[i]    
    counter = Counter(tokens)
    words_count = len(tokens)    
    for token in np.unique(tokens):        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))        
        tf_idf[doc, token] = tf*idf
    doc += 1

In [34]:
len(tf_idf)

30870

In [35]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [36]:
D = np.zeros((N, total_vocab_size))
for i in tf_idf:
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass

In [37]:
def gen_vector(tokens):
    Q = np.zeros((len(total_vocab)))
    counter = Counter(tokens)
    words_count = len(tokens)
    query_weights = {}
    for token in np.unique(tokens):   
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log((N+1)/(df+1))
        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [38]:
def cosine_similarity(query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    print("")
    #print(tokens)
    d_cosines = []
    query_vector = gen_vector(tokens)
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
    return d_cosines

In [39]:
Arr1=np.array(cosine_similarity(query[0]))
Arr2=np.array(cosine_similarity(query[1]))
Arr3=np.array(cosine_similarity(query[2]))
Arr4=np.array(cosine_similarity(query[3]))

In [40]:
Arr1=np.sort(Arr1)[::-1]
Arr2=np.sort(Arr2)[::-1]
Arr3=np.sort(Arr3)[::-1]
Arr4=np.sort(Arr4)[::-1]

In [41]:
print(Arr1)

[0.05914158 0.03724629 0.03537235 0.03494568 0.03190127 0.02899576
 0.02232992 0.01793591 0.01260646 0.01095667 0.00977259 0.00761484
 0.0050974  0.00034792 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]


In [42]:
print(Arr2)

[0.17361538 0.11749816 0.10196374 0.09628295 0.09400704 0.07020909
 0.05904454 0.05748283 0.04685354 0.04548307 0.04521942 0.04521942
 0.04521942 0.04521942 0.04521942 0.04438323 0.04297346 0.03443382
 0.03247782 0.03042162 0.03019721 0.0279758  0.02097895 0.02014976
 0.01607328 0.01554388 0.01500259 0.01201115 0.0108066  0.01009432
 0.00919175 0.00806413 0.0058309  0.00547935 0.00535529 0.00461734
 0.00229788 0.00214646 0.00059342 0.        ]


In [43]:
print(Arr3)

[0.55935501 0.55935501 0.55935501 0.55935501 0.55935501 0.36243975
 0.31176193 0.19518365 0.17731083 0.14234353 0.03680619 0.02819297
 0.01376431 0.01339877 0.01058761 0.01025469 0.00916198 0.00868684
 0.00644752 0.00615694 0.00498216 0.00464664 0.00356542 0.00321147
 0.00320244 0.00315377 0.00259173 0.00207528 0.00159614 0.00155535
 0.00151263 0.00143951 0.00119572 0.00094755 0.         0.
 0.         0.         0.         0.        ]


In [44]:
print(Arr4)

[0.30515465 0.30195095 0.23974559 0.15934495 0.15915839 0.13675317
 0.06437574 0.04725936 0.04517847 0.04101147 0.02878921 0.02633624
 0.02361456 0.02361456 0.02361456 0.02361456 0.02361456 0.00835428
 0.00714306 0.00574055 0.00530965 0.00256087 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
